#Dear chan (DRCH novel) and my **GPTQCustomLORA**
Main Article (RU): https://habr.com/ru/articles/751972/

Creator: https://github.com/CodeDruidX

Src text: https://github.com/CodeDruidX/Everlasting-Summer-txt

Fork from: https://github.com/qwopqwop200/gptqlora

Origin: https://github.com/artidoro/qlora

In [ ]:
!git clone https://github.com/qwopqwop200/gptqlora
!pip install auto-gptq
!pip install accelerate
!pip install evaluate
!pip install transformers==4.31.0
!pip install peft==0.4.0

In [ ]:
#%%capture mutedtoo
from huggingface_hub import snapshot_download
snapshot_download(repo_id="gurgutan/ruGPT-13B-4bit",local_dir=r"/ruGPT-3.5")

In [1]:
# @markdown Your function to create datasets.Dataset
def prepare_dataset():
  from transformers import TextDataset
  from datasets import Dataset
  with open("/gptqlora/data.txt","r",encoding="utf-8") as f:
    l=f.readlines()
  #l=l.replace("\n","\n\n")
  import random

  def get_random_substring(input_string, length):
      start = random.randrange(0, len(input_string) - length + 1)
      return input_string[start : start + length]
  #print(get_random_substring(l,500))
  l2=list(set(["\n".join(get_random_substring(l,random.randint(4,6))) for i in range(2000)]))
  print(len(l2),len(set(l2)))
  l = list(map(lambda x: {
              'input': '',
              'output': x
          },l2))
  print(l2)
  dataset=Dataset.from_list(l)
  return dataset

In [2]:
import sys
sys.argv=['gptqlora.py', '–learning_rate', '0.0001', '--model_path', '/ruGPT-3.5', '--max_steps', '300', '--dataset', 'CUSTOM-BABY', '--per_device_train_batch_size', '1', '--logging_steps', '5', '--save_steps', '20', '--output_dir', './output2']

In [ ]:
#@title GPTQCustomLora
# This source code is licensed under the MIT license


from collections import defaultdict
import copy
import json
import os
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging

import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LineByLineTextDataset,
    set_seed,
    Seq2SeqTrainer,
    LlamaTokenizerFast
)
from datasets import load_dataset
import evaluate

from peft import (
    LoraConfig,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftModel
)
from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from auto_gptq.utils.peft_utils import get_gptq_peft_model, GPTQLoraConfig
from auto_gptq import AutoGPTQForCausalLM
from auto_gptq.nn_modules.qlinear import GeneralQuantLinear

torch.backends.cuda.matmul.allow_tf32 = True

logger = logging.getLogger(__name__)

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def prepare_model_for_int8_training(model, use_gradient_checkpointing=True):
    r"""
    This method wraps the entire protocol for preparing a model before running a training. This includes:
        1- Cast the layernorm in fp32 2- making output embedding layer require grads 3- Add the upcasting of the lm
        head to fp32

    Args:
        model, (`transformers.PreTrainedModel`):
            The loaded model from `transformers`
    """
    for name, param in model.named_parameters():
        # freeze base model's layers
        param.requires_grad = False

    if use_gradient_checkpointing:
        # For backward compatibility
        if hasattr(model, "enable_input_require_grads"):
            model.enable_input_require_grads()
        else:

            def make_inputs_require_grad(module, input, output):
                output.requires_grad_(True)

            model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

        # enable gradient checkpointing for memory efficiency
        model.gradient_checkpointing_enable()

    return model

@dataclass
class ModelArguments:
    model_path: Optional[str] = field(
        default="./llama-7b/"
    )
    trust_remote_code: Optional[bool] = field(
        default=False,
        metadata={"help": "Enable unpickling of arbitrary code in AutoModelForCausalLM#from_pretrained."}
    )

@dataclass
class DataArguments:
    eval_dataset_size: int = field(
        default=1024, metadata={"help": "Size of validation dataset."}
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    source_max_len: int = field(
        default=1024,
        metadata={"help": "Maximum source sequence length. Sequences will be right padded (and possibly truncated)."},
    )
    target_max_len: int = field(
        default=256,
        metadata={"help": "Maximum target sequence length. Sequences will be right padded (and possibly truncated)."},
    )
    dataset: str = field(
        default='alpaca',
        metadata={"help": "Which dataset to finetune on. See datamodule for options."}
    )

@dataclass
class TrainingArguments(transformers.Seq2SeqTrainingArguments):
    cache_dir: Optional[str] = field(
        default=None
    )
    train_on_source: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to train on the input in addition to the target text."}
    )
    mmlu_split: Optional[str] = field(
        default='eval',
        metadata={"help": "The MMLU split to run on"}
    )
    mmlu_dataset: Optional[str] = field(
        default='mmlu-fs',
        metadata={"help": "MMLU dataset to use: options are `mmlu-zs` for zero-shot or `mmlu-fs` for few shot."}
    )
    do_mmlu_eval: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to run the MMLU evaluation."}
    )
    max_mmlu_samples: Optional[int] = field(
        default=None,
        metadata={"help": "If set, only evaluates on `max_mmlu_samples` of the MMMLU dataset."}
    )
    mmlu_source_max_len: int = field(
        default=2048,
        metadata={"help": "Maximum source sequence length for mmlu."}
    )
    full_finetune: bool = field(
        default=False,
        metadata={"help": "Finetune the entire model without adapters."}
    )
    adam8bit: bool = field(
        default=False,
        metadata={"help": "Use 8-bit adam."}
    )
    lora_r: int = field(
        default=64,
        metadata={"help": "Lora R dimension."}
    )
    lora_alpha: float = field(
        default=16,
        metadata={"help": " Lora alpha."}
    )
    lora_dropout: float = field(
        default=0.0,
        metadata={"help":"Lora dropout."}
    )
    max_memory_MB: int = field(
        default=24000,
        metadata={"help": "Free memory per gpu."}
    )
    report_to: str = field(
        default='none',
        metadata={"help": "To use wandb or something else for reporting."}
    )
    output_dir: str = field(default='./output', metadata={"help": 'The output dir for logs and checkpoints'})
    optim: str = field(default='paged_adamw_32bit', metadata={"help": 'The optimizer to be used'})
    per_device_train_batch_size: int = field(default=1, metadata={"help": 'The training batch size per GPU. Increase for better speed.'})
    gradient_accumulation_steps: int = field(default=16, metadata={"help": 'How many gradients to accumulate before to perform an optimizer step'})
    max_steps: int = field(default=10000, metadata={"help": 'How many optimizer update steps to take'})
    weight_decay: float = field(default=0.0, metadata={"help": 'The L2 weight decay rate of AdamW'}) # use lora dropout instead for regularization if needed
    learning_rate: float = field(default=0.0002, metadata={"help": 'The learnign rate'})
    remove_unused_columns: bool = field(default=False, metadata={"help": 'Removed unused columns. Needed to make this codebase work.'})
    max_grad_norm: float = field(default=0.3, metadata={"help": 'Gradient clipping max norm. This is tuned and works well for all models tested.'})
    gradient_checkpointing: bool = field(default=True, metadata={"help": 'Use gradient checkpointing. You want to use this.'})
    do_train: bool = field(default=True, metadata={"help": 'To train or not to train, that is the question?'})
    lr_scheduler_type: str = field(default='constant', metadata={"help": 'Learning rate schedule. Constant a bit better than cosine, and has advantage for analysis'})
    warmup_ratio: float = field(default=0.03, metadata={"help": 'Fraction of steps to do a warmup for'})
    logging_steps: int = field(default=10, metadata={"help": 'The frequency of update steps after which to log the loss'})
    group_by_length: bool = field(default=True, metadata={"help": 'Group sequences into batches with same length. Saves memory and speeds up training considerably.'})
    save_strategy: str = field(default='steps', metadata={"help": 'When to save checkpoints'})
    save_steps: int = field(default=250, metadata={"help": 'How often to save a model'})
    save_total_limit: int = field(default=40, metadata={"help": 'How many checkpoints to save before the oldest is overwritten'})

@dataclass
class GenerationArguments:
    # For more hyperparameters check:
    # https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
    # Length arguments
    max_new_tokens: Optional[int] = field(
        default=256,
        metadata={"help": "Maximum number of new tokens to be generated in evaluation or prediction loops"
                          "if predict_with_generate is set."}
    )
    min_new_tokens : Optional[int] = field(
        default=None,
        metadata={"help": "Minimum number of new tokens to generate."}
    )

    # Generation strategy
    do_sample: Optional[bool] = field(default=False)
    num_beams: Optional[int] = field(default=1)
    num_beam_groups: Optional[int] = field(default=1)
    penalty_alpha: Optional[float] = field(default=None)
    use_cache: Optional[bool] = field(default=True)

    # Hyperparameters for logit manipulation
    temperature: Optional[float] = field(default=1.0)
    top_k: Optional[int] = field(default=50)
    top_p: Optional[float] = field(default=1.0)
    typical_p: Optional[float] = field(default=1.0)
    diversity_penalty: Optional[float] = field(default=0.0)
    repetition_penalty: Optional[float] = field(default=1.0)
    length_penalty: Optional[float] = field(default=1.0)
    no_repeat_ngram_size: Optional[int] = field(default=0)

def find_all_linear_names(args, model):
    cls = GeneralQuantLinear if not(args.full_finetune) else torch.nn.Linear
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


class SavePeftModelCallback(transformers.TrainerCallback):
    def save_model(self, args, state, kwargs):
        print('Saving PEFT checkpoint...')
        if state.best_model_checkpoint is not None:
            checkpoint_folder = os.path.join(state.best_model_checkpoint, "adapter_model")
        else:
            checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)

    def on_save(self, args, state, control, **kwargs):
        self.save_model(args, state, kwargs)
        return control

    def on_train_end(self, args, state, control, **kwargs):
        def touch(fname, times=None):
            with open(fname, 'a'):
                os.utime(fname, times)

        touch(join(args.output_dir, 'completed'))
        self.save_model(args, state, kwargs)

def get_accelerate_model(args, checkpoint_dir):

    n_gpus = torch.cuda.device_count()
    max_memory = f'{args.max_memory_MB}MB'
    max_memory = {i: max_memory for i in range(n_gpus)}

    if args.full_finetune: assert args.bits in [16, 32]

    print(f'loading base model {args.model_path}...')
    model = AutoGPTQForCausalLM.from_quantized(
        args.model_path,
        low_cpu_mem_usage=True,
        device_map='auto',
        max_memory=max_memory,
        trust_remote_code=args.trust_remote_code,
        inject_fused_attention = True,
        inject_fused_mlp = False,
        use_triton=True,
        warmup_triton=False,
        trainable=True
    )
    model.model.quantize_config = model.quantize_config
    model.train()

    setattr(model, 'model_parallel', True)
    setattr(model, 'is_parallelizable', True)
    #modules = find_all_linear_names(args, model)

    model.config.torch_dtype=(torch.float32 if args.fp16 else (torch.bfloat16 if args.bf16 else torch.float32))

    if not args.full_finetune:
        model = prepare_model_for_int8_training(model, use_gradient_checkpointing=args.gradient_checkpointing)
    if args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    config = GPTQLoraConfig(
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        #target_modules=modules,
        lora_dropout=args.lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    if not args.full_finetune:
        if checkpoint_dir is not None:
            print("Loading adapters from checkpoint.")
            model = PeftModel.from_pretrained(model, join(checkpoint_dir, 'adapter_model'))
            for name, p in model.named_parameters():
                if 'lora' in name:
                    print(name, p.sum())
        else:
            print(f'adding LoRA modules...')
            model = get_gptq_peft_model(model, config, auto_find_all_linears=True, train_mode=True)

    if args.gradient_checkpointing:
        if hasattr(model, "enable_input_require_grads"):
            model.enable_input_require_grads()
        else:
            def make_inputs_require_grad(module, input, output):
                output.requires_grad_(True)
            model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)


    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if args.bf16:
                module = module.to(torch.bfloat16)
        if 'norm' in name:
            module = module.to(torch.float32)
        if 'lm_head' in name or 'embed_tokens' in name:
            if hasattr(module, 'weight'):
                if args.bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)
    return model

def print_trainable_parameters(args, model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    try:
        trainable_params /= (32//model.quantize_config.bits)
    except:
        pass
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable: {100 * trainable_params / all_param}")

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg

@dataclass
class DataCollatorForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer
    source_max_len: int
    target_max_len: int
    train_on_source: bool
    predict_with_generate: bool

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        # Extract elements
        sources = [example['input'] for example in instances]
        targets = [f"{example['output']}{self.tokenizer.eos_token}" for example in instances]
        # Tokenize
        tokenized_sources_with_prompt = self.tokenizer(
            sources,
            max_length=self.source_max_len,
            truncation=True,
        )
        tokenized_targets = self.tokenizer(
            targets,
            max_length=self.target_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        # Build the input and labels for causal LM
        input_ids = []
        labels = []
        for tokenized_source, tokenized_target in zip(
            tokenized_sources_with_prompt['input_ids'],
            tokenized_targets['input_ids']
        ):
            if not self.predict_with_generate:
                input_ids.append(torch.tensor(tokenized_source + tokenized_target))
                if not self.train_on_source:
                    labels.append(
                        torch.tensor([IGNORE_INDEX for _ in range(len(tokenized_source))] + copy.deepcopy(tokenized_target))
                    )
                else:
                    labels.append(torch.tensor(copy.deepcopy(tokenized_source + tokenized_target)))
            else:
                input_ids.append(torch.tensor(tokenized_source))
        # Apply padding
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX) if not self.predict_with_generate else None
        data_dict = {
            'input_ids': input_ids,
            'attention_mask':input_ids.ne(self.tokenizer.pad_token_id),
        }
        if labels is not None:
            data_dict['labels'] = labels
        return data_dict

def extract_unnatural_instructions_data(examples, extract_reformulations=False):
    out = {
        'input': [],
        'output': [],
    }
    for example_instances in examples['instances']:
        for instance in example_instances:
            out['input'].append(instance['instruction_with_input'])
            out['output'].append(instance['output'])
    if extract_reformulations:
        for example_reformulations in examples['reformulations']:
            if example_reformulations is not None:
                for instance in example_reformulations:
                    out['input'].append(instance['instruction_with_input'])
                    out['output'].append(instance['output'])
    return out

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response: "
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response: "
    ),
}

def extract_alpaca_dataset(example):
    if example.get("input", "") != "":
        prompt_format = PROMPT_DICT["prompt_input"]
    else:
        prompt_format = PROMPT_DICT["prompt_no_input"]
    return {'input': prompt_format.format(**example)}


def get_last_checkpoint(checkpoint_dir):
    if isdir(checkpoint_dir):
        is_completed = exists(join(checkpoint_dir, 'completed'))
        if is_completed: return None, True # already finished
        max_step = 0
        for filename in os.listdir(checkpoint_dir):
            if isdir(join(checkpoint_dir, filename)) and filename.startswith('checkpoint'):
                max_step = max(max_step, int(filename.replace('checkpoint-', '')))
        if max_step == 0: return None, is_completed # training started, but no checkpoint
        checkpoint_dir = join(checkpoint_dir, f'checkpoint-{max_step}')
        print(f"Found a previous checkpoint at: {checkpoint_dir}")
        return checkpoint_dir, is_completed # checkpoint found!
    return None, False # first training

def train():
    hfparser = transformers.HfArgumentParser((
        ModelArguments, DataArguments, TrainingArguments, GenerationArguments
    ))
    model_args, data_args, training_args, generation_args, extra_args = \
        hfparser.parse_args_into_dataclasses(return_remaining_strings=True)
    training_args.generation_config = transformers.GenerationConfig(**vars(generation_args))
    args = argparse.Namespace(
        **vars(model_args), **vars(data_args), **vars(training_args)
    )
    print(args)

    checkpoint_dir, completed_training = get_last_checkpoint(args.output_dir)
    if completed_training:
        print('Detected that training was already completed!')

    model = get_accelerate_model(args, checkpoint_dir)
    training_args.skip_loading_checkpoint_weights=True

    resume_from_checkpoint = checkpoint_dir
    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            checkpoint_dir, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_path = os.path.join(
                checkpoint_dir, "adapter_model"
            )

            checkpoint_name = os.path.join(
                checkpoint_path, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    model.config.use_cache = False
    print_trainable_parameters(args, model)
    print('loaded model')
    set_seed(args.seed)

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_path,
        cache_dir=args.cache_dir,
        padding_side="right",
        use_fast=True,
    )

    if tokenizer.pad_token is None:
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
            tokenizer=tokenizer,
            model=model,
        )

    if isinstance(tokenizer, LlamaTokenizerFast):
        # LLaMA tokenizer may not have correct special tokens set.
        # Check and add them if missing to prevent them from being parsed into different tokens.
        # Note that these are present in the vocabulary.
        # Note also that `model.config.pad_token_id` is 0 which corresponds to `<unk>` token.
        tokenizer.add_special_tokens(
            {
                "eos_token": tokenizer.convert_ids_to_tokens(model.config.eos_token_id),
                "bos_token": tokenizer.convert_ids_to_tokens(model.config.bos_token_id),
                "unk_token": tokenizer.convert_ids_to_tokens(model.config.pad_token_id),
            }
        )

    data_module = make_data_module(tokenizer=tokenizer, args=args)
    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
    )

    # Callbacks
    if not args.full_finetune:
        trainer.add_callback(SavePeftModelCallback)
    if args.do_mmlu_eval:
        if args.mmlu_dataset == 'mmlu-zs':
            mmlu_dataset = load_dataset("json", data_files={
                'eval': 'data/mmlu/zero_shot_mmlu_val.json',
                'test': 'data/mmlu/zero_shot_mmlu_test.json',
            })
            mmlu_dataset = mmlu_dataset.remove_columns('subject')
        # MMLU Five-shot (Eval/Test only)
        elif args.mmlu_dataset == 'mmlu' or args.mmlu_dataset == 'mmlu-fs':
            mmlu_dataset = load_dataset("json", data_files={
                'eval': 'data/mmlu/five_shot_mmlu_val.json',
                'test': 'data/mmlu/five_shot_mmlu_test.json',
            })
            # mmlu_dataset = mmlu_dataset.remove_columns('subject')
        mmlu_dataset = mmlu_dataset[args.mmlu_split]
        if args.max_mmlu_samples is not None:
            mmlu_dataset = mmlu_dataset.select(range(args.max_mmlu_samples))
        abcd_idx = [
            tokenizer("A", add_special_tokens=False).input_ids[0],
            tokenizer("B", add_special_tokens=False).input_ids[0],
            tokenizer("C", add_special_tokens=False).input_ids[0],
            tokenizer("D", add_special_tokens=False).input_ids[0],
        ]
        accuracy = evaluate.load("accuracy")

        class MMLUEvalCallback(transformers.TrainerCallback):
            def on_evaluate(self, args, state, control, model, **kwargs):
                data_loader = trainer.get_eval_dataloader(mmlu_dataset)
                source_max_len = trainer.data_collator.source_max_len
                trainer.data_collator.source_max_len = args.mmlu_source_max_len
                trainer.model.eval()
                preds, refs = [], []
                loss_mmlu = 0
                for batch in tqdm(data_loader, total=len(data_loader)):
                    (loss, logits, labels) = trainer.prediction_step(trainer.model,batch,prediction_loss_only=False,)
                    # There are two tokens, the output, and eos token.
                    for i, logit in enumerate(logits):
                        label_non_zero_id = (batch['labels'][i] != -100).nonzero()[0][0]
                        logit_abcd = logit[label_non_zero_id-1][abcd_idx]
                        preds.append(torch.argmax(logit_abcd).item())
                    labels = labels[labels != IGNORE_INDEX].view(-1, 2)[:,0]
                    for label in labels.tolist():
                        if label in abcd_idx:
                            refs += [abcd_idx.index(label)]

                    loss_mmlu += loss.item()
                # Extract results by subject.
                results = {'mmlu_loss':loss_mmlu/len(data_loader)}
                subject = mmlu_dataset['subject']
                subjects = {s:{'refs':[], 'preds':[]} for s in set(subject)}
                for s,p,r in zip(subject, preds, refs):
                    subjects[s]['preds'].append(p)
                    subjects[s]['refs'].append(r)
                subject_scores = []
                for subject in subjects:
                    subject_score = accuracy.compute(
                        references=subjects[subject]['refs'],
                        predictions=subjects[subject]['preds']
                    )['accuracy']
                    results[f'mmlu_{args.mmlu_split}_accuracy_{subject}'] = subject_score
                    subject_scores.append(subject_score)
                results[f'mmlu_{args.mmlu_split}_accuracy'] = np.mean(subject_scores)
                trainer.log(results)
                trainer.data_collator.source_max_len = source_max_len

        trainer.add_callback(MMLUEvalCallback)

    # Verifying the datatypes.
    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)

    all_metrics = {"run_name": args.run_name}
    # Training
    if args.do_train:
        train_result = trainer.train(resume_from_checkpoint=resume_from_checkpoint)
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        all_metrics.update(metrics)
    # Evaluation
    if args.do_eval:
        logger.info("*** Evaluate ***")
        metrics = trainer.evaluate(metric_key_prefix="eval")
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)
        all_metrics.update(metrics)
    # Prediction
    if args.do_predict:
        logger.info("*** Predict ***")
        prediction_output = trainer.predict(test_dataset=data_module['predict_dataset'],metric_key_prefix="predict")
        prediction_metrics = prediction_output.metrics
        predictions = prediction_output.predictions
        predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
        predictions = tokenizer.batch_decode(
            predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        with open(os.path.join(args.output_dir, 'predictions.jsonl'), 'w') as fout:
            for i, example in enumerate(data_module['predict_dataset']):
                example['prediction_with_input'] = predictions[i].strip()
                example['prediction'] = predictions[i].replace(example['input'], '').strip()
                fout.write(json.dumps(example) + '\n')
        print(prediction_metrics)
        trainer.log_metrics("predict", prediction_metrics)
        trainer.save_metrics("predict", prediction_metrics)
        all_metrics.update(prediction_metrics)

    if (args.do_train or args.do_eval or args.do_predict):
        with open(os.path.join(args.output_dir, "metrics.json"), "w") as fout:
            fout.write(json.dumps(all_metrics))


def make_data_module(tokenizer: transformers.PreTrainedTokenizer, args) -> Dict:
    # From origin author:




    """
    Make dataset and collator for supervised fine-tuning.
    Datasets are expected to have the following columns: { `input`, `output` }

    Available datasets to be selected with `dataset` argument:
        - alpaca, 52002 examples
        - alpaca cleaned, 51942 examples
        - chip2 (OIG), 210289 examples
        - self-instruct, 82612 examples
        - hh-rlhf (Anthropic), 160800 examples
        - longform, 23.7k examples

    Coming soon:
        - unnatural instructions core, 66010 examples
        - unnatural instructions full, 240670 examples
        - alpaca-gpt4, 52002 examples
        - unnatural-instructions-gpt4, 9000 examples
        - oa-rlhf (OpenAssistant) primary message tree only, 9209 examples
        - oa-rlhf-assistant (OpenAssistant) all assistant  replies with ranking
        - supernatural-instructions, 69624 examples (same as paper with 100 ex/task more can be used)
        - flan (FLAN v2), up to 20M examples available

    Not Available:
        - vicuna, not released at the moment.
    """
    # From me:
    """
    Just keep 'CUSTOM-BABY' as arg and edit prepare_dataset()
    """












    # Load dataset.
    # Alpaca
    print(args.dataset)

    if args.dataset == 'CUSTOM-BABY':
      dataset=prepare_dataset()
      print(dataset)
      #train_dataset=LineByLineTextDataset(tokenizer=tokenizer,file_path="/gptqlora/datafromES.txt",block_size=64)
      #print(train_dataset.examples)
      #train_dataset.examples = list(map(lambda x: {
      #      'input': '',
      #      'output': x
      #  },train_dataset.examples))
    #elif args.dataset == 'alpaca':
    # Alpaca clean
    elif args.dataset == 'alpaca-clean':
        dataset = load_dataset("yahma/alpaca-cleaned")
        dataset = dataset.map(extract_alpaca_dataset, remove_columns=['instruction'])
    # Chip2
    elif args.dataset == 'chip2':
        dataset = load_dataset("laion/OIG", data_files='unified_chip2.jsonl')
        dataset = dataset.map(lambda x: {
            'input': x['text'].split('\n<bot>: ')[0].replace('<human>: ', ''),
            'output': x['text'].split('\n<bot>: ')[1],
        }, remove_columns=['text', 'metadata'])
    # Self Instruct
    elif args.dataset == 'self-instruct':
        dataset = load_dataset("yizhongw/self_instruct", name='self_instruct')
        for old, new in [["prompt", "input"], ["completion", "output"]]:
            dataset = dataset.rename_column(old, new)
    # Anthropic rlhf
    elif args.dataset == 'hh-rlhf':
        dataset = load_dataset("Anthropic/hh-rlhf")
        dataset = dataset.map(lambda x: {
            'input': '',
            'output': x['chosen']
        }, remove_columns=['chosen', 'rejected'])
        print(dataset)
    # LongForm
    elif args.dataset == 'longform':
        dataset = load_dataset("akoksal/LongForm")
    elif args.dataset == 'vicuna':
        raise NotImplementedError("Vicuna data was not released.")
    else:
        raise NotImplementedError(f"Dataset {args.dataset} not implemented yet.")

    # Split train/eval, reduce size
    if args.do_eval or args.do_predict:
        if 'eval' in dataset:
            eval_dataset = dataset['eval']
        else:
            print('Splitting train dataset in train and validation according to `eval_dataset_size`')
            dataset = dataset["train"].train_test_split(
                test_size=args.eval_dataset_size, shuffle=True, seed=42
            )
            eval_dataset = dataset['test']
        if args.max_eval_samples is not None and len(eval_dataset) > args.max_eval_samples:
            eval_dataset = eval_dataset.select(range(args.max_eval_samples))
        if args.group_by_length:
            eval_dataset = eval_dataset.map(lambda x: {'length': len(x['input']) + len(x['output'])})
    if args.do_train:
        #train_dataset = dataset['train']
        train_dataset = dataset
        if args.max_train_samples is not None and len(train_dataset) > args.max_train_samples:
            train_dataset = train_dataset.select(range(args.max_train_samples))
        if args.group_by_length:
            train_dataset = train_dataset.map(lambda x: {'length': len(x['input']) + len(x['output'])})

    data_collator = DataCollatorForCausalLM(
        tokenizer=tokenizer,
        source_max_len=args.source_max_len,
        target_max_len=args.target_max_len,
        train_on_source=args.train_on_source,
        predict_with_generate=args.predict_with_generate,
    )
    return dict(
        train_dataset=train_dataset if args.do_train else None,
        eval_dataset=eval_dataset if args.do_eval else None,
        predict_dataset=eval_dataset if args.do_predict else None,
        data_collator=data_collator
    )



train()

---
---
# ALL NEXT CODE IS NOT MATTER
---
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Test model with lora
import torch
from transformers import AutoTokenizer
import auto_gptq
from auto_gptq import AutoGPTQForCausalLM, get_gptq_peft_model
from auto_gptq.utils.peft_utils import GPTQLoraConfig
#import gradio as gr

model_name = 'fffrrt/ruGPT-3.5-13B-GPTQ'
model_basename = 'gptq_model-4bit-128g'

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized("gurgutan/ruGPT-13B-4bit",
        low_cpu_mem_usage=True,
        device_map='auto',
        trust_remote_code=True,
        inject_fused_attention = True,
        inject_fused_mlp = False,
        use_triton=True,
        warmup_triton=False,
        trainable=True)

#t=torch.load('saved_model.pth')
#model.load_state_dict(t['model_state_dict'])
# loaded_tokenizer = t['tokenizer']

In [20]:
!python gptqlora.py -–learning_rate 0.0001 --model_path /ruGPT-3.5 --max_steps 300 --dataset CUSTOM-BABY --per_device_train_batch_size 1 --logging_steps 5 --save_steps 20 --output_dir ./output2

2023-08-28 14:48:52.414767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
loading base model /ruGPT-3.5...
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py", line 672, in _get_config_dict
    resolved_config_file = cached_file(
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py", line 417, in cached_file
    resolved_file = hf_hub_download(
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 110, in _inner_fn
    validate_repo_id(arg_value)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 164, in validate_repo_id
    raise HFValidationError(
huggingface_hub.utils._validators.HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: '/ruGPT-3.5'.

During handling

In [ ]:
peft_config = GPTQLoraConfig(
    inference_mode=True,
)
model = get_gptq_peft_model(model, peft_config, '/content/drive/MyDrive/ckp/adapter_model/adapter_model')

In [ ]:
model.is_loaded_in_4bit=True

In [ ]:
t=torch.load('saved_model.pth',torch.device('cpu'))
model.load_state_dict(t['model_state_dict'])
tokenizer = t['tokenizer']

del t
import gc
gc.collect()

9

In [ ]:
cfg={
"K":40,
"P":0.98,
"temperature":0.6,
"uningram":4
}
def editcfg(name,val):
  print(name,val)
  global cfg
  cfg[name]=val

def gen(text,tokens=10):
  encoded_input = tokenizer(text, return_tensors='pt').to('cuda:0')
  print(encoded_input.input_ids[:,-1900:].shape)
  output = model.generate(
      input_ids=encoded_input.input_ids[:,-1900:],
      max_new_tokens=tokens,
      do_sample=True,
      top_k=cfg['K'],
      top_p=cfg['P'],
      temperature=cfg['temperature'],
      #num_beams=4,
      no_repeat_ngram_size=cfg['uningram'],
      pad_token_id=tokenizer.eos_token_id,
      # num_return_sequences=5,
      # do_sample=True
      #repetition_penalty = 1.04
  )

  return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
#@title UI
import gradio as gr
def complete_with_gpt(text):
    text=gen(text,30)
    return text

with gr.Blocks() as demo:
    gr.Markdown("# Ever novel")
    textbox = gr.Textbox(placeholder="Верни контекст на место!", value="123")
    btn = gr.Button("Ожидать")
    gr.Markdown("---")
    with gr.Row():
      K=gr.Number(label="top_K",value=cfg["K"],minimum=1)
      P=gr.Number(label="top_P",value=cfg["P"],minimum=0,maximum=1)
      T=gr.Number(label="Temperature",value=cfg["temperature"],minimum=0)
      U=gr.Number(label="Unique ngram len",value=cfg["uningram"],minimum=0)
      APPLY=gr.Button(value="Применить")

    APPLY.click(fn=lambda x: editcfg("K",int(x)),inputs=K)
    APPLY.click(fn=lambda x: editcfg("P",x),inputs=P)
    APPLY.click(fn=lambda x: editcfg("temperature",x),inputs=T)
    APPLY.click(fn=lambda x: editcfg("uningram",int(x)),inputs=U)
    btn.click(fn=complete_with_gpt, inputs=textbox, outputs=textbox, queue=False)

demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

torch.Size([1, 1])
torch.Size([1, 277])
torch.Size([1, 307])
torch.Size([1, 337])
torch.Size([1, 367])
torch.Size([1, 397])
torch.Size([1, 279])
torch.Size([1, 309])
torch.Size([1, 339])
torch.Size([1, 369])
torch.Size([1, 399])
K 40
P 0.98
uningram 4
temperature 0.2
torch.Size([1, 22])
torch.Size([1, 52])
torch.Size([1, 28])
torch.Size([1, 53])
torch.Size([1, 83])
torch.Size([1, 113])
torch.Size([1, 143])


In [ ]:
#!pip install peft
#!pip install bitsandbytes
#del peft
from importlib import reload
import peft
peft=reload(peft)
import bitsandbytes as bnb
#from peft import LoraConfig, get_peft_model,  prepare_model_for_kbit_training
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = peft.get_peft_model(model, config)

model = peft.prepare_model_for_kbit_training(model)

# Сохраним обучающие данные в .txt файл
train_path = 'train_dataset.txt'
#with open(train_path, "w") as f:
#    f.write(prompt)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)

# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned2", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=0.01, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    auto_find_batch_size=True,
    per_device_eval_batch_size=1,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=1, # to make "virtual" batch size larger
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)
trainer.train()
torch.save({
      'model_state_dict': model.state_dict(),
      'tokenizer': tokenizer
  }, 'saved_model.pth')

NameError: ignored

In [ ]:
q_model = trainer.quantize()

In [ ]:
gen('''Семён: Посмотри, он совсем не страшный.
Лена выглянула у меня из-за спины.
Лена: Не страшный…
Семён: Сейчас, подожди.
Я мягко освободился от её объятий и подошёл к совёнку.
Сначала казалось, что он испугается и улетит, выпустив воланчик.
Однако совёнок, похоже, и не собирался двигаться с места.
Мне удалось схватить воланчик и аккуратно отобрать его у птицы.
Семён: Смотри, он совсем ручной! Хочешь его погладить?
Ленка взяла меня за руку.
Мы осторожно приблизились к совёнку, который'''.replace("\n","\n\n"),20)

torch.Size([1, 142])


'Семён: Посмотри, он совсем не страшный.\n\nЛена выглянула у меня из-за спины.\n\nЛена: Не страшный…\n\nСемён: Сейчас, подожди.\n\nЯ мягко освободился от её объятий и подошёл к совёнку.\n\nСначала казалось, что он испугается и улетит, выпустив воланчик.\n\nОднако совёнок, похоже, и не собирался двигаться с места.\n\nМне удалось схватить воланчик и аккуратно отобрать его у птицы.\n\nСемён: Смотри, он совсем ручной! Хочешь его погладить?\n\nЛенка взяла меня за руку.\n\nМы осторожно приблизились к совёнку, который в это время начал клевать что-то у себя под ногами.\n\nДойдя до него'

In [ ]:
trainer_load_from_checkpoint("12123123")

In [ ]:
gen("""from collections import defau""",20)

torch.Size([1, 6])


'from collections import defauxt\n This source code is licensed under the MIT license found in the\n'

In [ ]:
model.save_pretrained('./trained_model3/',use_safetensors=True)
tokenizer.save_pretrained('./trained_model3/')

In [ ]:
!sudo swapon /newswap

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
gc.collect()
torch.cuda.empty_cache()print(gen(prompt,30))


In [ ]:
encoded_input = tokenizer(prompt, return_tensors='pt').to('cuda:0')
print(encoded_input.input_ids.shape)
#print(len(encoded_input[0]))

In [ ]:
prompt="""Мне снился сон…
Казалось, я нахожусь в каком-то вакууме, а вокруг – пустота.
Но не только _вокруг_ – Я единственное существо во Вселенной.
Как будто она вернулась в некое сингулярное состояние перед самым Большим Взрывом.
И вот-вот что-то должно было произойти.
Вдруг я начал слышать голос.
Слов не разобрать, но он показался мне знакомым.
Голос что-то нежно нашёптывал, как будто убаюкивая.
И тут я понял... Это был голос той Незнакомки из автобуса. Той девушки из сна.
Мысли: Но что она хочет мне сказать? Кто она?..
Я проснулся.
Яркие лучи солнца били в глаза.
Время приближалось к полудню.
Лениво потянувшись и зевнув, я начал вспоминать вчерашний день.
За несколько секунд все его события пронеслись перед глазами: автобус, лагерь, местные обитатели.
Мысли: Но ведь всё не так, неправильно!
Не эта ситуация, не моё положение – они неправильны априори, – а моё отношение к ним.
Мысли: Ведь я вчера запросто заснул здесь, а до этого мило беседовал с местными пионерами, даже умудрялся шутить!
Мысли: Но как можно себя так вести в подобной ситуации?!
Мысли: Я же должен бояться, шарахаться от каждого шороха, избегать любого контакта с потенциально враждебными существами.
Все события прошедшего дня словно заволокло похмельной дымкой.
Мысли: Это очень похоже на утро после хорошей пьянки: вчерашнее естественное, непредосудительное, в высшей степени нормальное поведение утром становится кошмаром, гротескной гравюрой из иллюстраций к «Божественной комедии».
Мысли: Да, всё именно так, однако прошлого уже не вернуть.
Хотя, возможно, оценив обстановку, я действовал по ситуации.
Я огляделся по сторонам, пытаясь понять, не забросило ли меня куда-нибудь ещё, но домик Ольги Дмитриевны выглядел так же, как и вчера.
Всё было как будто на своих местах, разве что на спинке кровати висела пионерская форма.
Я с недоверием покрутил её в руках, примерил и оделся.
Мысли: Всё равно это лучше, чем ходить в зимней одежде.
Мысли: Посмотреть бы теперь на себя – наверняка выгляжу как клоун!
А для этого нужно зеркало. Хотя бы самое маленькое.
Нашлось оно на дверце шкафа.
Семён: Твою!..
Я взглянул на новоиспечённого пионера и аж отпрыгнул в сторону от неожиданности!
На другой стороне зеркала стоял какой-то подросток!
Похожий на меня, но не я!
Куда пропали недельная небритость, мешки под глазами, сутулость и смертельно уставшее выражение лица?!
Похоже, меня не закинули назад во времени или в параллельную реальность, а просто поменяли с кем-то телами.
Мысли: Действительно просто! Такое же на каждом шагу встречается!
Я пригляделся к незнакомцу повнимательнее и только тогда понял, что это я сам!
Только образца конца школы – начала института.
Мысли: Ладно, хотя бы так.
Мысли: Да уж, _человек в стрессовой ситуации_ слона и не приметил!
Мысли: А вот вожатая обратила внимание и вчера ночью меня отчитала за неподобающее обращение к ней…
Мысли: К чёрту!
Мысли: Вряд ли мой внешний вид влияет на что-то ещё.
Если верить часам, завтрак уже давно позади.
Мысли: Ну ладно, попробую всё же в столовой что-нибудь найти.
Мысли: Вчера же со Славей получилось.
От этих воспоминаний я невольно улыбнулся.
На улице ярко светило солнце, дул лёгкий ветерок.
Мысли: Прекрасный летний день.
Я уже несколько лет не чувствовал себя по утрам так хорошо.
Все проблемы на секунду улетели куда-то далеко, растворились в редких, цвета первого снега облаках.
Вдруг передо мной словно из ниоткуда появилась Ольга Дмитриевна.
Ольга Дмитриевна: Доброе утро, Семён!
Семён: Доброе!
Я улыбнулся, пытаясь всем своим видом показать, что несмотря ни на что утро моё было действительно добрым.
Ольга Дмитриевна: Ты только вчера приехал, так что будить я тебя не стала, но завтрак-то…
Ольга Дмитриевна: Хотя ладно! Вот, держи!
Она протянула мне бумажный свёрток.
Судя по масляным пятнам, внутри, скорее всего, бутерброды.
Семён: Ой, спасибо!
Ольга Дмитриевна: А теперь марш умываться!
Я уже собирался уходить.
Ольга Дмитриевна: Сейчас, подожди.
Ольга Дмитриевна забежала в домик и, вернувшись, сунула мне небольшой пакетик.
Внутри оказались зубная щётка, мыло, небольшое полотенце и что-то ещё – я особо не всматривался.
Ольга Дмитриевна: Пионер должен быть всегда чист и опрятен!
Ольга Дмитриевна: Дай я тебе галстук правильно завяжу на первый раз, а то он болтается. Потом научишься, сам будешь!
Семён: А может, не надо? Я сейчас умываться иду.
Мысли: Ну да, вдруг зацеплюсь за кран и удавлюсь...
Ольга Дмитриевна: Ладно, тогда потом. И не забудь про линейку.
Мысли: Карандаши, ручки, линейки… Такие вещи не забываются!
Семён: Какую линейку?
Ольга Дмитриевна: В смысле – какую линейку?!
Она нахмурилась.
Ольга Дмитриевна: Сегодня же понедельник!
Мысли: Странно, а по моим подсчётам – воскресенье…
Мысли: Впрочем, смена дня недели – это ещё не самое страшное.
Ольга Дмитриевна: Обычно у нас линейки рано утром, до завтрака, но сегодня понедельник, поэтому она будет в 12 часов.
Ольга Дмитриевна: Не опаздывай!
Семён: Хорошо. А где?
Ольга Дмитриевна: На площади, где же ещё!
Спорить было бессмысленно.
Я направился в сторону «помывочной».
На отдельный душ и туалет рассчитывать не приходилось, но при виде этого выкидыша загнивающего социализма – причудливой черепашки с панцирем из жести, множеством ног-кранов и кафельным брюшком – мне стало несколько не по себе.
Я не был брезгливым человеком, но тем не менее, стоя тут, понял, что всё же есть какой-то минимальный уровень привычного комфорта, без которого жить мне довольно проблематично.
Вот ведь как бывает – когда теряешь вещи, которые всегда казались совершенно обыденными и естественными, понимаешь, что на самом деле они были незаменимы.
Мысли: А, да и чёрт с ним! Выбирать всё равно не из чего.
Вода оказалась просто ледяной.
Если помыть руки не составило особого труда, то вот умыться или прополоскать рот ей – уже большая проблема.
В пакетике, который мне дала Ольга Дмитриевна, не нашлось зубной пасты.
Можно, конечно, было почистить зубы и так, но в полотенце была завернута какая-то кругленькая коробочка.
«Зубной порошок».
Мысли: Прелестно! +1 за то, что я где-то в прошлом.
Умылся я довольно быстро, в том числе и из-за ледяной воды.
Кто-то быстро шёл, даже бежал в мою сторону.
Я обернулся.
Передо мной стояла Славя в спортивном костюме.
Похоже, эта девочка будет хорошо выглядеть абсолютно во всём – и в пионерской форме, и в купальнике, и, наверное, даже в космическом скафандре.
Славя: Физкульт-привет!
Семён: Охай… То есть, бобр… Доброе утро! Вот…
Приветствие мне удалось выбрать не сразу.
Славя: Почему на завтрак не пришёл?
Семён: Проспал.
Я сказал это так, словно гордился своим достижением.
Семён: Но мне Ольга Дмитриевна бутерброды принесла.
Славя: А, ну отлично тогда! Не забудь про линейку!
Семён: Да, конечно.
Мысли: Забудешь тут.
Славя: Ладно, я побежала, не скучай!
Она помахала мне на прощание и скрылась за поворотом тропинки.
Мысли: Судя по всему, линейка начнётся через пару минут.
Стоило быстренько забежать «домой», закинуть пакетик с умывальными принадлежностями, съесть бутерброды и только уже потом идти на площадь.
Я распахнул дверь домика вожатой и вбежал внутрь так, как будто запрыгивал в последний вагон уходящего поезда.
Но, кажется, это было не лучшим решением – посреди комнаты стояла Ольга Дмитриевна…
И переодевалась!
""".replace("\n","\n\n")

In [ ]:
len(prompt)

7301

In [ ]:
seed=123

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

gen("Здравствуйте",20)

torch.Size([1, 1])


'Здравствуйте. помогите пожалуйста решить задачу. У меня есть 2 сестры, одна родная, а другая двоюродная.'

In [ ]:
text = prompt
while 1:
  text=next(text,30)
  print(text)
  text+=input("Вы:").replace("@","Семён:").replace(";;;","\n")
  text=text[-2000:]

In [1]:
%%capture muted
!pip install auto-gptq gradio
!git clone -b peft_integration https://github.com/PanQiWei/AutoGPTQ.git && cd AutoGPTQ
!pip install .[triton]
%cd ..
#!git clone https://github.com/timdettmers/bitsandbytes.git
!pip install bitsandbytes
!git clone https://github.com/qwopqwop200/gptqlora
%cd gptqlora
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install -r requirements.txt
!pip install protobuf==3.20.*
!pip install huggingface_hub
!pip uninstall peft
!pip install peft==0.4.0